In [ ]:
import os
# os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.7/bin") // path to nvidia dlls, must have CUDA drivers installed
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from keras.models import Sequential
from keras.layers import Dense
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import mixed_precision

### Loading Data

In [ ]:
fp = "../data/features_combined.csv"
batch_pd = pd.read_csv(fp, index_col=False)
dataset = batch_pd.copy()
dataset.sort_values(by=['policy'], ascending=True, inplace=True)

# dataset
dataset.isna().sum()
dataset = dataset.dropna().drop(columns=['policy', 'barcode'])

In [ ]:
## Global Model Config
EPOCHS = 2500
UNITS = 1
LEARNING_RATE = 0.01
CALLBACK = tf.keras.callbacks.EarlyStopping(monitor='mae', patience=15, min_delta=0.01)

## Data split
Split by policy fast charge first(5C - 8C), and then by policy slow charge (1C - 4C)


In [ ]:
normal_charge_dataset = dataset.iloc[0:29, :] # 29 data points
print(normal_charge_dataset.shape)

## Fast-Charge Test-Train split
Selecting alternate batteries for training and testing

In [ ]:
normal_charge_train_ds = normal_charge_dataset.iloc[0::2, :]
normal_charge_test_ds = normal_charge_dataset.iloc[1::2, :]

normalcharge_train_features = normal_charge_train_ds.copy()
normalcharge_test_features = normal_charge_test_ds.copy()

normal_train_labels = normalcharge_train_features.pop('cycle_life')
normal_test_labels = normalcharge_test_features.pop('cycle_life')

# Linear Regress
### Layering and Build Model

In [ ]:
# Normalize layer
QDiffLinVar = np.array(normalcharge_train_features['QDiffLinVar'])
QDiffLinVar_normalizer = layers.Normalization(input_shape=[1,], axis=None)
QDiffLinVar_normalizer.adapt(QDiffLinVar)

variance_model_normal_charge_traditional = tf.keras.Sequential([
    QDiffLinVar_normalizer,
    layers.Dense(UNITS, input_dim=1, activation='relu'),
    layers.Dense(1, activation='linear', dtype='float32', name='predictions')
])
variance_model_normal_charge_traditional.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mae']
    )


### Train the Model

In [ ]:
%%time
history_traditional = variance_model_normal_charge_traditional.fit(
    normalcharge_train_features['QDiffLinVar'],
    normal_train_labels,
    epochs=EPOCHS,
    verbose=2,
    # callbacks=[CALLBACK],
    validation_data=(normalcharge_test_features['QDiffLinVar'], normal_test_labels))

### Plot loss graph and Evaluate

In [ ]:
def plot_loss(hist_trad, hist_tl):
  plt.figure("Normal-charge base model loss", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.plot(np.sqrt(hist_trad.history['loss']), label='loss_traditional')
  plt.plot(np.sqrt(hist_trad.history['val_loss']), label='val_loss_traditional')
  plt.ylim([0, 800])
  plt.xlabel('Epoch')
  plt.ylabel('Error [cycles]')
  plt.legend()
  plt.grid(True)
  plt.title('Normal-charge base model loss')

plot_loss(history_traditional)

In [ ]:
test_results = {}
test_results['normal_charge_variance_model_traditional'] = variance_model_normal_charge_traditional.evaluate(
    normalcharge_test_features['QDiffLinVar'],
    normal_test_labels, verbose=1) #sqrt for mse

### Make Predictions

In [ ]:
def plot_prediction(y_train, y_test):
  plt.figure("Transfer Learning vs Traditional", figsize=(5,5), dpi=100, facecolor='w', edgecolor='k')
  plt.axes(aspect='equal')
  plt.scatter(y_train, normal_train_labels, label='Predictions (train)')
  plt.scatter(y_test, normal_test_labels, label='Predictions (test)')
  lims = [0, 2000]
  plt.xlim(lims)
  plt.ylim(lims)
  plt.plot(lims, lims, 'k', )
  plt.xlabel('Predicted Cycle life')
  plt.ylabel('Actual Cycle life')
  plt.legend()

normal_train_prediction2 = variance_model_normal_charge_traditional.predict(normal_charge_train_ds['QDiffLinVar'])
normal_test_prediction2 = variance_model_normal_charge_traditional.predict(normal_charge_test_ds['QDiffLinVar'])
plot_prediction(normal_train_prediction2, normal_test_prediction2)

### Final Results and conclusions

In [ ]:
test_results['normal_charge_variance_model_traditional'][0] = test_results['normal_charge_variance_model_traditional'][0] ** 0.5
pd.DataFrame(test_results, index=['RMSE', 'MAE']).T